In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
featuresDf = pd.read_pickle('./dataset/featuresDF_1_LN.pkl')
featuresDf['fourier_mfcc'] = [np.concatenate([featuresDf.fourier[i],
                              featuresDf.mfcc[i]]) for i in range(len(featuresDf))]

In [3]:
X = np.array(featuresDf['fourier_mfcc'].tolist())
y = np.array(featuresDf['class'].tolist())

In [ ]:
# GradientBoostingClassifier GridSearchCV:
# Default: learning_rate=0.1, n_estimators=100
parameters = { 
    'n_estimators': [200, 400],
    'learning_rate' :[0.1, 0.3]}
gbc = GradientBoostingClassifier() 
clf = GridSearchCV(gbc, parameters, cv=5, scoring='accuracy', verbose=5, n_jobs= -1)
clf.fit(X, y)
print(clf.best_estimator_)
print(clf.best_score_) 
print(clf.best_params_)
# Refits an estimator using the best found parameters on the whole dataset.
from sklearn.externals import joblib
joblib_file = "./models/GradientBoostingClassifier_GSCV.pkl"
joblib.dump(clf, joblib_file)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
